In [1]:
!pip install huggingface
!pip install datasets

   ---------------------------------------- 0.0/510.5 kB ? eta -:--:--
   ------- -------------------------------- 92.2/510.5 kB 2.6 MB/s eta 0:00:01
   ----------- ---------------------------- 143.4/510.5 kB 2.8 MB/s eta 0:00:01
   -------------------- ------------------- 256.0/510.5 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 510.5/510.5 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/170.9 kB ? eta -:--:--
   --------------------------------------- 170.9/170.9 kB 10.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/388.9 kB ? eta -:--:--
   ----------------------------------- --- 358.4/388.9 kB 11.2 MB/s eta 0:00:01
   --------------------------------------- 388.9/388.9 kB 12.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
    ---

In [2]:
from datasets import *

In [22]:
dataset = load_dataset("cc100","gn")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset cc100 downloaded and prepared to C:/Users/matth/.cache/huggingface/datasets/cc100/gn/1.0.0/f0eb694c41a28a8c093b9db6d024dc2186ada300bb33d1d149bdf0481b68ad16. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [40]:
len(dataset['train']['text'])

72285

Let's start by figuring out what our possible character sets are between a sample of this data, and the Guarani data in the training set.

In [26]:
import pandas as pd

In [93]:
guarani_gold = pd.read_csv("../data/yoyodyne/guarani-train.tsv", sep='\t')


In [94]:
# Let's try to make a quick heuristic comparison between our datasets

In [165]:
max([len(sent) for sent in guarani_gold.iloc[:,1]])

70

In [167]:
max_guarani_sent_len = max([len(sent) for sent in guarani_gold.iloc[:,0]])

In [191]:
min_guarani_sent_len = min([len(sent) for sent in guarani_gold.iloc[:,1]])

In [192]:
min_guarani_sent_len

9

In [95]:
all_text_gold = " ".join(guarani_gold.iloc[:,0]) + " ".join(guarani_gold.iloc[:,1])
all_text_cc = " ".join(dataset['train']['text'])

In [96]:
all_text_cc[1000:1100]

"pa umi kuatiahai apohare. Upéi ojehechaukákuri Mbo'ehára Rafael Dendia ha Juan Ramón Jiménez rembias"

In [97]:
all_text_gold[400:500]

'ke pokã Peẽ napejapómi ogyke pokã Peẽ napejapómi ogyke pokã Peẽ napejapómi ogyke pokã Ko po ojupi Ko'

In [98]:
# as a first approximation, let's compare the character sets

In [99]:
gold_character_set = set([char for char in all_text_gold])
cc_character_set = set([char for char in all_text_cc])

In [100]:
unique_to_gold = gold_character_set - cc_character_set
len(list(unique_to_gold))

0

In [101]:
unique_to_cc = cc_character_set - gold_character_set
len(list(unique_to_cc))

825

Okay, so it seems that the CC data is basically a character superset of the gold standard data we have.

Let's check some most common character n-grams in the gold standard data and see if we can find similar words in the CC-100 data.

In [197]:
def build_ngrams(full_text, n=3):
    n_grams = []
    split_text = full_text.split()
    for word in split_text:
        if len(word) >= n:
            n_grams.extend([word[i:i+n] for i in range(len(word) - n + 1)])
    
    return n_grams
        

In [103]:
# let's also normalize the text to lower case

all_text_gold = all_text_gold.lower()
all_text_cc = all_text_cc.lower()

In [104]:
from collections import Counter

gold_ngrams = build_ngrams(all_text_gold)

gold_ngram_count = Counter(gold_ngrams)


In [105]:
cc_ngrams = build_ngrams(all_text_cc)

cc_ngram_count = Counter(cc_ngrams)

In [114]:
gold_top_ngrams = set([x[0] for x in gold_ngram_count.most_common(100)])

cc_top_ngrams = set([x[0] for x in cc_ngram_count.most_common(100)])


In [115]:
gold_unique = gold_top_ngrams - cc_top_ngrams
len(gold_unique)



60

In [116]:
cc_unique = cc_top_ngrams - gold_top_ngrams
len(cc_unique)

60

In [109]:
df = pd.DataFrame.from_dict({"CC": list(sorted(cc_unique)), "Gold": list(sorted(gold_unique))})

In [110]:
df

,CC,Gold
0,a'e,are
1,and,boj
2,apo,bot
3,apy,che
4,ara,dom
5,ava,end
6,ave,ere
7,ba’,gyk
8,cha,ha’
9,e'ẽ,hín


In [118]:
#what about what they have in common?

common_ngrams = gold_top_ngrams - gold_unique

In [120]:
common_ngrams

{'agu',
 'aku',
 'ape',
 'apy',
 'are',
 'a’e',
 'eku',
 'emb',
 'emo',
 'end',
 'ere',
 'ete',
 'gua',
 'ha’',
 'ich',
 'iko',
 'iku',
 'jap',
 'kua',
 'kur',
 'kué',
 'mbi',
 'mbo',
 'mby',
 'nda',
 'nde',
 'ndu',
 'ogu',
 'ohe',
 'omb',
 'omo',
 'ret',
 'umi',
 'upé',
 'uri',
 'uér',
 'ára',
 'éra',
 'ñan',
 'ñem'}

In [121]:
def contains_foreign_character(text):
    for char in text:
        if char not in gold_character_set:
            return True
        
    return False

In [122]:
# Let's try filtering out sentences in the CC data where foreign characters exist--that might at least get us to somewhere closer

cc_words = all_text_cc.split()
cc_okay_words = [word for word in cc_words if not contains_foreign_character(word)]
cc_text = " ".join(cc_okay_words)

In [123]:
cc_ngrams_cleaned = build_ngrams(cc_text)

cc_ngram_count_cleaned = Counter(cc_ngrams_cleaned)

In [124]:
cc_top_ngrams_cleaned = set([x[0] for x in cc_ngram_count_cleaned.most_common(100)])


cc_unique = cc_top_ngrams_cleaned - gold_top_ngrams
gold_unique = gold_top_ngrams - cc_top_ngrams_cleaned

df = pd.DataFrame.from_dict({"CC": list(sorted(cc_unique)), "Gold": list(sorted(gold_unique))})


In [133]:
common_ngrams = cc_top_ngrams_cleaned - cc_unique

In [134]:
common_ngrams

{'agu',
 'aku',
 'ape',
 'apy',
 'are',
 'asy',
 'a’e',
 'eku',
 'emb',
 'emo',
 'end',
 'ere',
 'ete',
 'gua',
 'ha’',
 'ich',
 'iko',
 'iku',
 'jap',
 'kua',
 'kur',
 'kué',
 'mbi',
 'mbo',
 'mby',
 'nda',
 'nde',
 'ndu',
 'ogu',
 'ohe',
 'omb',
 'omo',
 'ret',
 'umi',
 'upé',
 'uér',
 'ára',
 'éra',
 'ñan',
 'ñem'}

In [153]:
cc_words_with_and = [word for word in cc_okay_words if 'jap' in word]

gold_words_with_and = [word for word in all_text_gold.split(" ") if 'jap' in word]

In [154]:
sorted(list(set(gold_words_with_and)))

['najajapómi',
 'napejapói',
 'napejapómi',
 'ndajajapói',
 'nderejapói',
 'nderejapókuri',
 'nderejapómi',
 'ndojapói',
 'nojapopami',
 'nojapómi',
 'norojapómi',
 'ojapopa',
 'ojapopami',
 'ojapopata',
 'ojapopava’ekue',
 'pejapomi',
 'rejapo']

In [164]:
sorted(list(set([word for word in cc_okay_words if "japóm" in word])))

['ajapómi',
 'ojapóma',
 'ojapómahina',
 'ojapómakatu',
 'ojapómaneraka’e',
 'ojapómava',
 'ojapómi',
 'ojejapóma',
 'ojejapóma,',
 'ojejapómajepi',
 'ojejapómava',
 'ojejapómi',
 'ojejapómiva’erâ',
 'ojejapómiva’erã']

In [143]:
print("hembi’ukuéra" == "hembi’ukuéra")

True


In [157]:
#final reasonable sanity check -- are there wordforms in the original data that do not appear at all in the other corpus?

for word in all_text_gold.split(" "):
    if word not in cc_okay_words:
        print(f"{word}\n")


ombojerekuri

kutuhakuéra

ombojerekuri

kutuhakuéra

mombe’ukuéra

mombe’ukuéra

mombe’ukuéra

mombe’ukuéra

mombe’ukuéra

mombe’ukuéra

napejapómi

napejapómi

napejapómi

napejapómi

napejapómi

napejapómi

napejapómi

temirendukuéra

temirendukuéra

temirendukuéra

temirendukuéra

temirendukuéra

ndombovuta

ndombovuta

ndombovuta

ndombovuta

ndombovuta

apekuéra

apekuéra

apekuéra

apekuéra

apekuéra

naiñemoñaréi

naiñemoñaréi

naiñemoñaréi

naiñemoñaréi

naiñemoñaréi

naiñemoñaréi

nañemoingeta

nañemoingeta

nañemoingeta

nañemoingeta

nañemoingeta

ñeimo’ã’ỹva

ndojurata

aovevekuéra

ñeimo’ã’ỹva

ndojuramo’ãi

aovevekuéra

ñeimo’ã’ỹva

ndojurata

aovevekuéra

ñeimo’ã’ỹva

ndojuramo’ãi

aovevekuéra

ñeimo’ã’ỹva

ndojurata

aovevekuéra

ñeimo’ã’ỹva

ndojuramo’ãi

aovevekuéra

ñeimo’ã’ỹva

ndojurata

aovevekuéra

ñeimo’ã’ỹva

ndojuramo’ãi

aovevekuéra

ñeimo’ã’ỹva

ndojurata

aovevekuéra

ñeimo’ã’ỹva

ndojuramo’ãi

aovevekuéra

ajavými

iñeikotevẽva

akuápe

ajavými

iñeikotev

In [168]:
max_guarani_sent_len

92

In [181]:
guarani_data_filtered = dataset['train']['text']

In [182]:
guarani_data_filtered = [x.strip() for x in guarani_data_filtered]

In [184]:
guarani_data_filtered = [x for x in guarani_data_filtered if not contains_foreign_character(x)]

In [186]:
guarani_data_filtered = [x for x in guarani_data_filtered if len(x)<= max_guarani_sent_len * 1.5]

In [193]:
guarani_data_filtered = [x for x in guarani_data_filtered if len(x) > 5]

In [194]:
len(guarani_data_filtered)

6676

In [195]:
guarani_data_filtered[100:110]

['Tekotevẽ oñembo’e tekoha reheguáre opárupi ha opaichave.',
 'oñeñangareko haguã tekoháre, terã ojoko haguã tembiapo, ombyaiva terã ombyaiséva tekoha, ha',
 'porã mba’épepa ombyaí aiguéne tekohápe.',
 'oiméva, ’y opáva peguarã terã ikatúva opavave oiporu, avei umi ’y yvyguypegua umichagua',
 'ndivéi, ambue Tetãmini ha umi táva, ojererekóne tenondérupi umi aguĩgua.',
 'Ojeikove haguã oñekotevẽ katuete ’ýre.',
 'Tetãmini mburuvicha oñangareko va’erã opaite yjyvýre.',
 'Tetãmini ha táva mba’e.',
 'Tetãmini ha táva mokõive ikatuha peve ojapo omoingouka va’erã yvy jeporu ha avei',
 'Tembiapo tekotevẽva ojapo va’erã tavaregua omoporãve haguã tekove, ome’ẽ va’erã umi']

In [210]:
# some other cleanup 
guarani_data_filtered = [x.lower() for x in guarani_data_filtered]

In [211]:
unique_gold_ngrams = build_ngrams(all_text_gold.lower(), n=2)
unique_gold_ngrams = list(set(unique_gold_ngrams))

In [216]:
def contains_foreign_trigram(sentence):
    sent_ngrams = build_ngrams(sentence, n=2)
    for ngram in sent_ngrams:
        if ngram not in unique_gold_ngrams and '.' not in ngram:
            print(ngram)
            return True
    return False

In [220]:
import random

In [227]:
random.seed(2024)

In [231]:
guarani_data_filtered = list(set([x.capitalize() for x in guarani_data_filtered]))

In [236]:
len(guarani_data_filtered)

3516

In [234]:
sampled = random.sample(guarani_data_filtered, 500)

In [240]:
# make into some DF

df_dict = {"Source": sampled, "Target": sampled, "Change": ["NOCHANGE"]*len(sampled)}

df_to_save = pd.DataFrame.from_dict(df_dict)

In [241]:
df_to_save.to_csv("../data/augmented/guarani-identity-externalcorp.tsv", sep='\t', header=False, index=False)